<a href="https://colab.research.google.com/github/RubyNixx/Pop_Health_Streamlit/blob/main/POPULATION_HEALTH_BY_URBAN_EXTENTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# POPULATION HEALTH BY URBAN EXTENTS
Population health based on synthetic data, visualised using a streamlit python application.

You will need to upload 2 CSV files, provided in the GitHub Repo.

In [16]:
# Import python packages
!pip install streamlit pydeck h3 shapely

import pandas as pd
import h3
import pydeck as pdk
import streamlit as st
import json
from h3 import LatLngPoly, LatLngMultiPoly

## Curating the data

Firstly get the data - we are going to focus on the location, the Gender, the body weight and finally each morbidity.

In [13]:
from google.colab import files

uploaded = files.upload()
#UPLOAD 'DEFAULT_DATABASE.DEFAULT_SCHEMA.SYNTHETIC_POPULATION'


Saving URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_AREAS_TBL.csv to URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_AREAS_TBL.csv
Saving DEFAULT_DATABASE.DEFAULT_SCHEMA.SYNTHETIC_POPULATION.csv to DEFAULT_DATABASE.DEFAULT_SCHEMA.SYNTHETIC_POPULATION (1).csv


In [32]:
# Load Data
population_health = pd.read_csv('DEFAULT_DATABASE.DEFAULT_SCHEMA.SYNTHETIC_POPULATION.csv')
built_up_areas = pd.read_csv('URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_AREAS_TBL.csv')


# EDA

In [48]:
#View columns available
population_health.columns

Index(['Unnamed: 0', 'NHS_NUMBER', 'DATE_OF_BIRTH', 'FIRST_NAME', 'LAST_NAME',
       'HOUSEHOLD', 'AGE', 'PRACTICE_CODE', 'PRACTICE_NAME',
       'DISTANCE_FROM_PRACTICE', 'ESTABLISHMENT_TYPE',
       'EDUCATIONAL_ESTABLISHMENT_UPN', 'EDUCATIONAL_ESTABLISHMENT_NAME',
       'UNI_COURSE_CODE', 'UNI_COURSE_TITLE', 'LAT', 'LON',
       'MARITAL_STATUS_CODE', 'MARITAL_STATUS', 'GENERAL_HEALTH_CODE',
       'GENERAL_HEALTH', 'OCCUPATION_CODE', 'OCCUPATION', 'SEX_CODE', 'SEX',
       'HOURS_WORKED_CODE', 'HOURS_WORKED', 'ECONOMICACTIVITY_CODE',
       'ECONOMICACTIVITY', 'HOUSEHOLD_TYPE_CODE', 'HOUSEHOLD_TYPE',
       'ADDRESS_1', 'ADDRESS_2', 'ADDRESS_3', 'ADDRESS_4', 'ADDRESS_5',
       'ADDRESS_6', 'YEARS_AT_ADDRESS', 'POSTCODE', 'POSTCODE_NO_SPACE',
       'LOCAL_AUTHORITY_CODE', 'LOCAL_AUTHORITY', 'LSOA_CODE', 'ETHNICITY',
       'CANCER', 'DIABETES', 'COPD', 'ASTHMA', 'HYPERTENSION', 'MORBILITIES',
       'ICB22NM', 'ICB22CD', 'PCN', 'MULTIPLE_MOBILITIES', 'BODY_WEIGHT',
       'H3'],

In [50]:
# Distinct values in 'MARITAL_STATUS'
print(population_health['MARITAL_STATUS'].unique())

# Distinct values in 'OCCUPATION'
print(population_health['OCCUPATION'].unique())

['Single'
 'Divorced or formerly in a same-sex civil partnership which is now legally dissolved'
 ' Married or in a same sex civil partnership'
 'Widowed or surviving partner from same-sex civil partnership'
 'Seperated (but still legally married or still legally in a same-sex civil partnership)'
 '<16 years old therefore ineligible to marry']
['Administrative and secretarial occupations'
 'Sales and customer service occupations'
 'Managers, directors and senior officials'
 'Associate professional and technical occupations'
 'Skilled trades occupations' 'Process, plant and machine operatives'
 'Professional occupations' 'Elementary occupations'
 'Caring, Leisure and other service occupations' 'Not Working'
 '<16 years old']


In [52]:
#Rename a column

population_health = population_health.rename(columns={'MULTIPLE_MOBILITIES': 'MULTIPLE_MORBIDITIES'})

print(population_health.head(5))

   Unnamed: 0  NHS_NUMBER DATE_OF_BIRTH FIRST_NAME LAST_NAME  HOUSEHOLD  AGE  \
0           0    55984819    1976-05-21       Mary  Skerrett  H20263425   49   
1           1    27281573    1925-05-21     Rhonda  Friedman  H20263651  100   
2           2    28985199    1986-05-21       Gary  Montalvo  H20263613   39   
3           3    19386361    1964-05-21       Anne     Scott  H20263478   61   
4           4    42539555    1993-05-21      Susan  Eldridge  H20263665   32   

  PRACTICE_CODE PRACTICE_NAME  DISTANCE_FROM_PRACTICE  ... COPD  ASTHMA  \
0        P92030   DR SEABROOK              277.614209  ...  NaN     NaN   
1        P92030   DR SEABROOK              254.298787  ...  NaN     NaN   
2        P92030   DR SEABROOK              258.751208  ...  NaN     NaN   
3        P92030   DR SEABROOK              260.438865  ...  NaN     NaN   
4        P92030   DR SEABROOK              219.516524  ...  NaN     NaN   

  HYPERTENSION MORBILITIES                                       ICB

In [54]:
#Show by age

age_summary = (
    population_health
    .groupby('AGE')
    .agg(Total_Morbidities=('MULTIPLE_MORBIDITIES', 'sum'),
         Total_Population=('MULTIPLE_MORBIDITIES', 'count'))
    .reset_index()
    .sort_values('AGE')
)
print(age_summary)

     AGE  Total_Morbidities  Total_Population
0      0                  3                50
1      1                  2                28
2      2                  2                33
3      3                  5                31
4      4                  1                23
..   ...                ...               ...
99    99                  0                 4
100  100                  1                 6
101  101                  0                12
102  102                  5                10
103  103                  1                10

[104 rows x 3 columns]


In [55]:
# Filter by AGE ≥ 5

# Over 5
population_health_over5 = population_health[population_health['AGE'] >= 5]

# Under 5
population_health_under5 = population_health[population_health['AGE'] < 5][['NHS_NUMBER','FIRST_NAME','LAST_NAME','AGE']]

print(population_health_under5.head(10))


     NHS_NUMBER FIRST_NAME   LAST_NAME  AGE
103  9056942793       Ruth     Whitney    0
177    25207586     Milton  Williamson    4
255  9034823097      Graig       Davis    0
495  9368453639   Patricia    Mckenzie    0
496  9368453639   Patricia    Mckenzie    0
508    18480044    Cameron       Sabin    0
516    46202609     Edward       Ortiz    1
527    42040243       Anna       Ortiz    0
529     4858707    Krystal    Stephens    0
537    34819413  Claudette    Stephens    0


In [56]:
occupations = population_health['OCCUPATION'].dropna().unique()
# In Streamlit:
# selected_occupation = st.selectbox('Choose Occupation:', occupations)

In [57]:
#Filtered DataFrame by Occupation

selected_occupation = occupations[0]  # Replace with user selection in Streamlit

filtered_df = population_health[population_health['OCCUPATION'] == selected_occupation]

print(filtered_df.head(5))

    Unnamed: 0  NHS_NUMBER DATE_OF_BIRTH FIRST_NAME  LAST_NAME  HOUSEHOLD  \
0            0    55984819    1976-05-21       Mary   Skerrett  H20263425   
3            3    19386361    1964-05-21       Anne      Scott  H20263478   
6            6    28219522    1994-05-21      Shawn   Weishaar  H20263707   
10          10    24756033    1992-05-21    Raymond  Laliberte  H20263665   
16          16    23068360    1968-05-21   Danielle  Pellegrin  H20263665   

    AGE PRACTICE_CODE PRACTICE_NAME  DISTANCE_FROM_PRACTICE  ... COPD  ASTHMA  \
0    49        P92030   DR SEABROOK              277.614209  ...  NaN     NaN   
3    61        P92030   DR SEABROOK              260.438865  ...  NaN     NaN   
6    31        P92030   DR SEABROOK              286.628143  ...  NaN     NaN   
10   33        P92030   DR SEABROOK              219.516524  ...  NaN     NaN   
16   57        P92030   DR SEABROOK              219.516524  ...  NaN     NaN   

   HYPERTENSION MORBILITIES                       

In [58]:
#Group by marital status

marital_summary = (
    filtered_df
    .groupby('MARITAL_STATUS')
    .agg(
        Multiple_Morbidities=('MULTIPLE_MORBIDITIES', 'sum'),
        Cancer=('CANCER', 'sum'),
        Diabetes=('DIABETES', 'sum'),
        COPD=('COPD', 'sum'),
        Asthma=('ASTHMA', 'sum')
    )
    .reset_index()
)
print(marital_summary)


                                      MARITAL_STATUS  Multiple_Morbidities  \
0         Married or in a same sex civil partnership                    13   
1  Divorced or formerly in a same-sex civil partn...                     6   
2  Seperated (but still legally married or still ...                     1   
3                                             Single                    18   
4  Widowed or surviving partner from same-sex civ...                     7   

   Cancer  Diabetes  COPD  Asthma  
0     4.0      24.0   5.0    29.0  
1     1.0      10.0   1.0     7.0  
2     1.0       3.0   0.0     3.0  
3     1.0      19.0   1.0    35.0  
4     5.0      10.0   2.0     3.0  


In [59]:
pivot = pd.pivot_table(
    population_health,
    values='CANCER',
    index=['ICB22NM', 'PRACTICE_NAME'],
    columns='MARITAL_STATUS',
    aggfunc='sum',
    fill_value=0
)
print(pivot.head())


MARITAL_STATUS                                                              Married or in a same sex civil partnership  \
ICB22NM                                            PRACTICE_NAME                                                         
NHS Bedfordshire, Luton and Milton Keynes Integ... EASTGATE SURGERY                                                0.0   
                                                   GARDENIA PRACTICE                                               1.0   
                                                   THE GROVE SURGERY                                               2.0   
NHS Birmingham and Solihull Integrated Care Board  FRANKLEY HEALTH CENTRE                                          0.0   
                                                   NEW ROAD SURGERY                                                0.0   

MARITAL_STATUS                                                             <16 years old therefore ineligible to marry  \
ICB22NM                

In [60]:
import altair as alt

# For Altair, reset index so columns are available
pivot_reset = pivot.reset_index()

# Adjust column names as needed to match your data
chart = (
    alt.Chart(pivot_reset)
    .mark_circle()
    .encode(
        x="MARRIED OR IN A SAME SEX CIVIL PARTNERSHIP:Q",
        y="SINGLE:Q",
        size="SEPERATED (BUT STILL LEGALLY MARRIED OR STILL LEGALLY IN A SAME-SEX CIVIL PARTNERSHIP):Q",
        color="<16 YEARS OLD THEREFORE INELIGIBLE TO MARRY:N",
        tooltip=["PRACTICE_NAME:N"]
    )
)
chart.show()
# In Streamlit: st.altair_chart(chart, use_container_width=True)


alt.Chart(...)

In [61]:
import requests
import geopandas as gpd
import pandas as pd

url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LSOA_2011_Boundaries_Super_Generalised_Clipped_BSC_EW_V4/FeatureServer/0/query'
params = {
    "where": "1=1",
    "outFields": "*",
    "f": "geojson"
}

response = requests.get(url, params=params)
response.raise_for_status()

gdf = gpd.read_file(response.text)
gdf['GEOM'] = gdf['geometry'].apply(lambda x: x.wkb_hex)

output = pd.DataFrame({
    'LSOA11CD': gdf['LSOA11CD'],
    'GEOM': gdf['GEOM']
})

output.to_csv('lsoa_polygons.csv', index=False)


We will now bucket all the locations into **H3 indexes**.  Next, a new dataframe is created called **POPULATION_H3**.  

In [34]:
def get_h3(lat, lon, res=8):
    return h3.latlng_to_cell(lat, lon, res)

population_health['H3'] = population_health.apply(lambda row: get_h3(row['LAT'], row['LON'], 8), axis=1)


In [36]:
population_h3 = (
    population_health.groupby(['H3','BODY_WEIGHT','SEX'])
    .agg(
        TOTAL_POPULATION=('BODY_WEIGHT','count'),
        CANCER=('CANCER','sum'),
        DIABETES=('DIABETES','sum'),
        COPD=('COPD','sum'),
        ASTHMA=('ASTHMA','sum'),
        HYPERTENSION=('HYPERTENSION','sum'),
        LAT=('LAT','mean'),
        LON=('LON','mean')
    )
    .reset_index()
)

Now lets look at the built up urban areas.

In [38]:
# Display the first 5 rows
built_up_areas.head(5)

,Unnamed: 0,GSSCODE,NAME1_TEXT,NAME1_LANGUAGE,NAME2_TEXT,NAME2_LANGUAGE,AREAHECTARES,GEOMETRY_AREA_M,GEOMETRY,GEOGRAPHY
0,0,S45001606,Walkerburn,NaN,NaN,NaN,30.00,300000,"MULTIPOLYGON (((335725 637225, 335725 637250, ...","{\n ""coordinates"": [\n [\n [\n ..."
1,1,E63008454,Walkeringham,NaN,NaN,NaN,53.44,534375,"MULTIPOLYGON (((476850 392125, 476825 392125, ...","{\n ""coordinates"": [\n [\n [\n ..."
2,2,E63011204,Walkern,NaN,NaN,NaN,52.31,523125,"MULTIPOLYGON (((528700 225575, 528700 225600, ...","{\n ""coordinates"": [\n [\n [\n ..."
3,3,E63007847,Walkington,NaN,NaN,NaN,82.38,823750,"MULTIPOLYGON (((499525 436725, 499500 436725, ...","{\n ""coordinates"": [\n [\n [\n ..."
4,4,E63008456,Wallasey,NaN,NaN,NaN,1659.13,16591250,"MULTIPOLYGON (((324750 389450, 324750 389425, ...","{\n ""coordinates"": [\n [\n [\n ..."


Next we need to join the two datasets together.  we need to join by the H3 index code.  Therefore, we need to split out all the towns by the same H3 index number.  You can use the function **H3_COVERAGE_STRINGS** for this

In [40]:
def parse_geojson(geojson_str):
    return json.loads(geojson_str)

def geojson_to_h3shape(geojson_obj):
    if geojson_obj['type'] == 'Polygon':
        outer = [[lat, lon] for lon, lat in geojson_obj['coordinates'][0]]
        holes = []
        if len(geojson_obj['coordinates']) > 1:
            for hole in geojson_obj['coordinates'][1:]:
                holes.append([[lat, lon] for lon, lat in hole])
        return LatLngPoly(outer, *holes)
    elif geojson_obj['type'] == 'MultiPolygon':
        polys = []
        for poly_coords in geojson_obj['coordinates']:
            outer = [[lat, lon] for lon, lat in poly_coords[0]]
            holes = []
            if len(poly_coords) > 1:
                for hole in poly_coords[1:]:
                    holes.append([[lat, lon] for lon, lat in hole])
            polys.append(LatLngPoly(outer, *holes))
        return LatLngMultiPoly(*polys)
    else:
        raise ValueError(f"Unsupported geometry type: {geojson_obj['type']}")

def h3shape_to_cells(h3shape, resolution=8):
    return list(h3.h3shape_to_cells(h3shape, resolution))

built_up_areas['geojson'] = built_up_areas['GEOGRAPHY'].apply(parse_geojson)
built_up_areas['h3shape'] = built_up_areas['geojson'].apply(geojson_to_h3shape)
built_up_areas['H3'] = built_up_areas['h3shape'].apply(h3shape_to_cells)
coverage = (
    built_up_areas[['GSSCODE', 'NAME1_TEXT', 'H3']]
    .explode('H3')
    .reset_index(drop=True)
)
print(coverage.head(10))


     GSSCODE    NAME1_TEXT               H3
0  S45001606    Walkerburn              NaN
1  E63008454  Walkeringham  881942a197fffff
2  E63011204       Walkern  88194e48d5fffff
3  E63007847    Walkington  88194058d3fffff
4  E63008456      Wallasey  8819510e0bfffff
5  E63008456      Wallasey  8819510e5dfffff
6  E63008456      Wallasey  8819510e03fffff
7  E63008456      Wallasey  8819510e1dfffff
8  E63008456      Wallasey  8819510e07fffff
9  E63008456      Wallasey  8819510e51fffff


You will note that there are multiple coverage strings per row.  We select the result which is returned as **VALUE** as well as all the original columns.  We will only select **GSSCODE** and **NAME1_TEXT**.  The Value Column is renamed as **H3**.

In [41]:
population_by_area = pd.merge(coverage, population_h3, on='H3', how='inner')

## Visualising the data


### Creating Filters

Firstly, let's create a filters to the dataset.  It would be good to filter by urban name, body weight and sex.  This is where we create streamlit components.  We will create three drop downlists.  I have nested it inside a container with a pre defined height and chose to create the drop down lists in 3 columns.

Each select box is populated with distinct values from the newly created dataframe.

In [42]:
import streamlit as st

st.title('POPULATION HEALTH BY URBAN AREA')

# Filters
col1, col2, col3 = st.columns(3)
with col1:
    urban_area = st.selectbox('Select Urban Area:', population_by_area['NAME1_TEXT'].unique())
with col2:
    body_weight = st.selectbox('Select Body Weight:', population_by_area['BODY_WEIGHT'].unique())
with col3:
    SEX = st.selectbox('Select Gender:', population_by_area['SEX'].unique())

df = population_by_area[
    (population_by_area['NAME1_TEXT'] == urban_area) &
    (population_by_area['BODY_WEIGHT'] == body_weight) &
    (population_by_area['SEX'] == SEX)
]

st.dataframe(df)

2025-05-26 21:30:53.415 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:30:53.512 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-26 21:30:53.513 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:30:53.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:30:53.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:30:53.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:30:53.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:30:53.522 Thread 'MainThread': mi

DeltaGenerator()

### Create Metrics
Now we have fitered to the right areas, let's now summarize the results and create metrics.  Again, used the columns to lay out all the metrics accross the page

In [43]:

# Metrics
pop_metrics = df.agg({
    'TOTAL_POPULATION':'sum',
    'CANCER':'sum',
    'DIABETES':'sum',
    'COPD':'sum',
    'ASTHMA':'sum',
    'HYPERTENSION':'sum'
})

col1, col2, col3, col4, col5, col6 = st.columns(6)
col1.metric('Total Population', pop_metrics['TOTAL_POPULATION'])
col2.metric('Cancer Sufferers', pop_metrics['CANCER'])
col3.metric('Diabetics', pop_metrics['DIABETES'])
col4.metric('COPD Sufferers', pop_metrics['COPD'])
col5.metric('Asthmatics', pop_metrics['ASTHMA'])
col6.metric('Hypertension', pop_metrics['HYPERTENSION'])

2025-05-26 21:31:01.355 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:31:01.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:31:01.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:31:01.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:31:01.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:31:01.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:31:01.361 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:31:01.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

### Creating a map

You will be creating a H3 map to visualise the hexagons which can filter by each extent area.  NB the color is in RGB format, you will need to manipulate the RGB based on the fields in the data.  This example is using the total population field.  Further calculations might need to be considered to have further control of how the colours are presented.

In [44]:
# Map
if not df.empty:
    LAT = df['LAT'].mean()
    LON = df['LON'].mean()
    layer = pdk.Layer(
        "H3HexagonLayer",
        df,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon="H3",
        get_fill_color="[255 - TOTAL_POPULATION, 255-TOTAL_POPULATION, 255]",
        get_line_color=[1, 1, 1],
        line_width_min_pixels=1,
    )
    view_state = pdk.ViewState(latitude=LAT, longitude=LON, zoom=12, bearing=0, pitch=0)
    r = pdk.Deck(
        map_style=None,
        layers=[layer],
        initial_view_state=view_state,
        tooltip={
            "html": "Total Population: {TOTAL_POPULATION}<br>"
                    "Total Cancer: {CANCER}<br>"
                    "Total Diabetes: {DIABETES}<br>"
                    "Total COPD: {COPD}<br>"
                    "Total Hypertension: {HYPERTENSION}"
        }
    )
    st.pydeck_chart(r)

2025-05-26 21:31:09.058 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:31:09.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 21:31:09.069 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


# Switching to Streamlit to view the dashboard

##Option 1: You can use pyngrok to tunnel the Streamlit app from Colab to the web.
This is not for production, but good for demos.

After running, click the printed URL to view your app.

In [47]:
!pip install streamlit pyngrok

import os
from pyngrok import ngrok

# Write your Streamlit script to a file
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
st.title("Hello Streamlit from Colab!")
""")  # Replace with your actual script content

# Start Streamlit in the background
os.system('streamlit run app.py &')

# Get a public URL via ngrok
public_url = ngrok.connect(port='8501')
print('Streamlit app available at:', public_url)


ERROR:pyngrok.process.ngrok:t=2025-05-26T21:38:42+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-26T21:38:42+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-26T21:38:42+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

## Option 2 - If You’re On Your Own Computer

Download your app script (e.g., app.py) to your computer.

Open a terminal/command prompt on your computer.

Navigate to the folder with app.py.


Run:

```
#bash
streamlit run app.py
```

Visit http://localhost:8501 in your browser.